In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1. CONFIGURACIÓN ---
FILE_NAME = "lab5.txt"
FS = 1000  # Frecuencia de muestreo en Hz
HEADER_LINES = 70 # Número de líneas de cabecera a saltar (hasta EndOfHeader)

# Definición de los segmentos de tiempo en segundos
SEGMENTS = [
    (0, 1 * 60),    # 0 a 60 segundos
    (1 * 60, 2 * 60), # 60 a 120 segundos
    (2 * 60, 4 * 60), # 120 a 240 segundos
    (4 * 60, 6 * 60), # 240 a 360 segundos
    (6 * 60, None)    # 360 segundos hasta el final
]

# --- 2. CARGAR Y PROCESAR DATOS ---
try:
    # Usamos skiprows para saltar la cabecera completa. 
    # La columna 5 (índice 5) corresponde a la señal 'A4'.
    data_signal = np.loadtxt(FILE_NAME, skiprows=HEADER_LINES)[:, 5]
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    print("Asegúrate de que 'lab5.txt' esté en el mismo directorio.")
    exit()

# Crear el vector de tiempo (t) para todo el dataset
t_full = np.arange(len(data_signal)) / FS

print(f"Datos cargados. Total de muestras: {len(data_signal)}")

# --- 3. GRAFICACIÓN POR SEGMENTOS ---
# Creamos una figura con 5 subplots apilados
fig, axes = plt.subplots(5, 1, figsize=(14, 15), sharex=False)

for i, (start_time, end_time) in enumerate(SEGMENTS):
    # Convertir tiempo a índices de muestra (muestra de inicio y fin)
    start_sample = start_time * FS
    
    if end_time is not None:
        end_sample = end_time * FS
        segment_label = f"{start_time//60} min a {end_time//60} min"
    else:
        # Si end_time es None, va hasta el final de los datos
        end_sample = len(data_signal)
        segment_label = f"{start_time//60} min al Final"

    # Extraer el segmento de tiempo y el segmento de señal usando slicing de NumPy
    t_segment = t_full[start_sample:end_sample]
    signal_segment = data_signal[start_sample:end_sample]

    # Ajuste para que el tiempo del segmento empiece en 0 si lo desea,
    # pero es mejor mantener el tiempo absoluto para referencia.
    # Si quieres que empiece en 0: t_segment = t_segment - t_segment[0] 

    # Graficar en el subplot correspondiente
    ax = axes[i]
    ax.plot(t_segment, signal_segment, linewidth=1, color='darkblue')
    
    # Configuración de los ejes
    ax.set_title(f"Segmento {i+1}: {segment_label} (Muestras: {len(signal_segment)})", 
                 fontsize=14, fontweight="bold")
    ax.set_xlabel("Tiempo [s]")
    ax.set_ylabel("Amplitud [unidades]") # Las unidades dependen del sensor (A4)
    ax.grid(True, linestyle="--", alpha=0.7)

# Mejorar el espaciado entre subplots
plt.tight_layout()
plt.show()